# Introducción

En este cuaderno se va a presentar el apartado en el que el proyecto comienza a utilizar las librerías python de pandas y también el uso de Scikit-Learn:

##### Normalización del documento
* 1.- Cargar los archivos .csv de las pruebas recogidas, tanto benignas como malignas.
* 2.- Realizar la unión de ambos archivos en uno solo, añadiendo una columna final con valores "0" para tráfico benigno y "1" para tráfico maligno
* 3.- Eliminar columnas innecesarias que producen ruido
* 4.- Completar el archivo final para el estudio realizando la normalización usando MinMaxScaler()

##### Aprendizaje y resultados de precisión
* 5.- Importación de librerias
* 6.- Se asignan archivos de trabajo y se divide el conjunto de datos
* 7.- Se crea y entrena el modelo
* 8.- Se muestra la precisión del modelo

# Normalización del documento

Se realiza la importación de librerías en primera instancia

In [22]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import ipaddress
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error, classification_report


## Cargar archivos

En este primer paso, se procede a cargar los archivos generados tras la ejecución de Dorothea tanto con la opción "-t attack" como con "-t normal"

Se generan una serie de interacciones en el sistemas que permiten generar archivos .csv con todo el flujo de datos de red necesario para el estudio.

Tras generar dichos .csv, estos se cargan de la siguiente manera:

In [23]:
# Ruta al archivo CSV
maligno = "/home/osboxes/Documents/Dorothea/Labs/lab_attacks/results/Pruebas/Maligno.csv"
benigno = "/home/osboxes/Documents/Dorothea/Labs/lab_normal/results/Benigno.csv"


## Modificación y unión de archivos

A continuación se agregan los valores "0" para tráfico benigno y "1" para tráfico maligno en los archivos .csv anteriormente indicados. Para ello, en primer lugar se crean dos variables con dichos valores para posteriormente agregarlos a los archivos .csv cargados en una nueva columna denominada "Tipo Trafico". Se finaliza este paso unificando ambos archivos:

In [24]:
# Valor fijo a añadir en la nueva columna
valor_maligno = 1
valor_benigno = 0

# Cargar archivos CSV
df1 = pd.read_csv(maligno)
df2 = pd.read_csv(benigno)
dftest = pd.read_csv(test)

# Agregar una columna con un valor fijo
df1['Tipo Trafico'] = valor_maligno
df2['Tipo Trafico'] = valor_benigno

# Se unen los archivos en uno
df_concatenado = pd.concat([df1, df2], axis=0)

## Eliminación de columnas innecesarias

Para una mejor gestión de los datos y una mejor comprensión, se proceden a eliminar las columnas que introducen ruido en las muestras

In [25]:
# Se eliminan columnas innecesarias
columnas_eliminar = ['#:unix_secs', 'unix_nsecs', 'sysuptime', 'exaddr', 'first', 'last', 'engine_type', 'engine_id',  'nexthop', 'src_mask', 'dst_mask', 'src_as', 'dst_as' ]  
df_concatenado = df_concatenado.drop(columnas_eliminar, axis=1)


## Gestión de las columnas que contienen direcciones IP

Para normalizar las columnas del archivo .csv que tienen formato de dirección IP, en primer lugar se transforman dichas columnas a un formato con el cual se pueda trabajar una vez que se aplique la normalización posterior.
En este caso se procede con la transformación a enteros utilizando int(ipaddress.IPv4Address(ip))

In [26]:
def ip_to_integer(ip):
	return int(ipaddress.IPv4Address(ip))

# Se definen las columnas del archivo que tienen las direcciones IP que se tienen que normalizar
DireccionesIP = ["srcaddr", "dstaddr"]
for direccion in DireccionesIP:
    df_concatenado[direccion] = df_concatenado[direccion].apply(ip_to_integer)

## Archivo normalizado

Para finalizar este primer apartado, se procede con la normalización del documento a través de la función MinMaxScaler() facilitada por la libreria Scikit-Learn.
De esta manera, se realizará el entrenamiento de nuestra inteligencia con valores entre 0 y 1, permitiendo así, que las pruebas realizadas no sufran errores.

In [27]:
# Se realiza ahora la normalización se las columnas para 
# posteriormente trabajar con ellas
columnas = ['dpkts', 'doctets', 'srcaddr', 'dstaddr', 'input', 'output', 'srcport', 'dstport', 'prot', 'tos', 'tcp_flags', 'Tipo Trafico']
scaler = preprocessing.Normalizer(norm='l2', copy=True)
scaler = scaler.fit(df_concatenado)
scaler = MinMaxScaler()
df_concatenado[columnas] = scaler.fit_transform(df_concatenado[columnas])
df_concatenado.to_csv('/home/osboxes/Documents/Dorothea/Scripts/csv_normalizado.csv', index=False)


## Aprendizaje y resultados de precisión

### En segunda instancia se trabaja con algortimos de aprendizaje automática e inteligencia artificial

Se asigna el archivo .csv, se facilitan los datos para el estudio y se divide el conjunto de datos

In [28]:
X = df_concatenado.drop('Tipo Trafico', axis=1)  # Coge todos los datos menos los de la columna Tipo Trafico
y = df_concatenado['Tipo Trafico'] #Valor para el estudio

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## MLP Classifier

Para finalizar, se crea y entrena el modelo MLP para posteriomente evaluarlo y obtener los valores objetivo

In [29]:
# Crear y entrenar el modelo MLP
mlp2 = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000)  # Ajusta los tamaños de las capas ocultas según tus necesidades
mlp2.fit(X_train, y_train)

# Evaluar el modelo
accuracy = mlp2.score(X_test, y_test)
print('Precisión del modelo:', accuracy)

# Realizar predicciones sobre los datos de prueba
y_pred = mlp2.predict(X_test)

# Obtener la matriz de confusión
confusion_mat = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:', confusion_mat)

# Obtener el informe de clasificación
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:', classification_rep)

